In [1]:
import json
import sqlite3
import pandas as pd
import config
import numpy as np
import requests

from sqlalchemy import create_engine
engine = create_engine('postgresql://'+config.db['user']+':'+config.db['password']+config.db['url'])
##--- SET CONFIG SOURCE
sSource = "api"
if sSource=="file":
    with open('./data/shortsales01-02-23.json','r') as f:
        data = json.loads(f.read())
elif sSource=="api":
    r = requests.get('https://www.set.or.th/api/set/shortsales')
    data = r.json()
    if r.status_code!=200:
        exit

cdate = data['fromDate'][:10]
#d1.info()
print(cdate)
df = pd.json_normalize(data['shortSales'])

df.info()
df.rename(columns = {'symbol': 'series', 'percentValue': 'per', 'percentVolume': 'perc'}, inplace = True)
df['trddate'] = data['fromDate'][:10] #pd.to_datetime('today').date() - pd.Timedelta("1 day")
#df.info()
print(df)

2023-02-02
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   symbol         418 non-null    object 
 1   volume         418 non-null    float64
 2   value          418 non-null    float64
 3   percentVolume  418 non-null    float64
 4   percentValue   418 non-null    float64
dtypes: float64(4), object(1)
memory usage: 16.5+ KB
     series      volume        value       perc        per     trddate
0       BCT       800.0      47200.0  72.727273  72.727273  2023-02-02
1       BEM   6899300.0   67374710.0  33.107476  33.107573  2023-02-02
2     SUPER  12866800.0    8234752.0  30.495612  30.408120  2023-02-02
3    EGATIF    174900.0    1189320.0  29.593909  29.601053  2023-02-02
4     BANPU  14887400.0  172693840.0  27.234577  27.229860  2023-02-02
..      ...         ...          ...        ...        ...         ...
413    RS-R      3400.0    

In [2]:
sql = "select * from sshortsell where trddate = '"+ df['trddate'][0].strftime('%Y-%m-%d') + "'"
print(sql)
rst = pd.read_sql(sql, con=engine)
print(len(rst.axes[0]))
if len(rst.axes[0]) == 0:df.to_sql(name='sshortsell', con=engine, if_exists='append')
print(df)

select * from sshortsell where trddate = '2023-02-02'
205
     series      volume        value       perc        per     trddate
0       BCT       800.0      47200.0  72.727273  72.727273  2023-02-02
1       BEM   6899300.0   67374710.0  33.107476  33.107573  2023-02-02
2     SUPER  12866800.0    8234752.0  30.495612  30.408120  2023-02-02
3    EGATIF    174900.0    1189320.0  29.593909  29.601053  2023-02-02
4     BANPU  14887400.0  172693840.0  27.234577  27.229860  2023-02-02
..      ...         ...          ...        ...        ...         ...
413    RS-R      3400.0      56860.0   0.037879   0.037846  2023-02-02
414   BTG-R      1700.0      56975.0   0.041363   0.041130  2023-02-02
415   JTS-R       900.0      44100.0   0.016071   0.016187  2023-02-02
416    HL-R       300.0       7320.0   0.011467   0.011191  2023-02-02
417  BAFS-R       200.0       6625.0   0.008820   0.008772  2023-02-02

[418 rows x 6 columns]
